# Python deltalake documentation

This notebook contains the code snippets used in the Python deltalake documentation.

In [24]:
import pathlib

import deltalake as dl
import pandas as pd
from deltalake import DeltaTable, write_deltalake
from tabulate import tabulate

In [27]:
import polars as pl

df = pl.DataFrame({"num": [1, 2, 3], "letter": ["a", "b", "c"]})
df.write_delta("~/tmp/cool/some-table")

## Create Delta table

In [25]:
df = pd.DataFrame({"num": [1, 2, 3], "letter": ["a", "b", "c"]})

In [26]:
write_deltalake("~/tmp/cool/some-table", df)

OSError: Encountered object with invalid path: Error parsing Path "/Users/matthew.powers/Documents/code/delta/delta-examples/notebooks/python-deltalake/~/tmp/cool/some-table": Encountered illegal character sequence "~" whilst parsing path segment "~"

In [4]:
write_deltalake("tmp/some-table", df)

In [5]:
dt = DeltaTable("tmp/some-table")

In [6]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|     1 | a        |
|     2 | b        |
|     3 | c        |
+-------+----------+


In [7]:
!tree tmp/some-table

tmp/some-table
├── 0-62dffa23-bbe1-4496-8fb5-bff6724dc677-0.parquet
└── _delta_log
    └── 00000000000000000000.json

1 directory, 2 files


In [8]:
!jq . tmp/some-table/_delta_log/00000000000000000000.json

{
  "protocol": {
    "minReaderVersion": 1,
    "minWriterVersion": 1
  }
}
{
  "metaData": {
    "id": "b96ea1a2-1830-4da2-8827-5334cc6104ed",
    "name": null,
    "description": null,
    "format": {
      "provider": "parquet",
      "options": {}
    },
    "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"num\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"letter\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns": [],
    "createdTime": 1701740315599,
    "configuration": {}
  }
}
{
  "add": {
    "path": "0-62dffa23-bbe1-4496-8fb5-bff6724dc677-0.parquet",
    "size": 2208,
    "partitionValues": {},
    "modificationTime": 1701740315597,
    "dataChange": true,
    "stats": "{\"numRecords\": 3, \"minValues\": {\"num\": 1, \"letter\": \"a\"}, \"maxValues\": {\"num\": 3, \"letter\": \"c\"}, \"nullCount\": {\"num\": 0, \"letter\": 0}}"
  }
}
{
  "commitInfo": {
    "timestamp": 1701740315602,
    "operation": "CREATE 

## Append

In [13]:
df = pd.DataFrame({"num": [8, 9], "letter": ["dd", "ee"]})

In [10]:
write_deltalake("tmp/some-table", df, mode="append")

In [14]:
dt = DeltaTable("tmp/some-table")

In [15]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|     1 | a        |
|     2 | b        |
|     3 | c        |
|     8 | dd       |
|     9 | ee       |
+-------+----------+


In [16]:
!tree tmp/some-table

tmp/some-table
├── 0-62dffa23-bbe1-4496-8fb5-bff6724dc677-0.parquet
├── 1-57abb6fb-2249-43ba-a7be-cf09bcc230de-0.parquet
└── _delta_log
    ├── 00000000000000000000.json
    └── 00000000000000000001.json

1 directory, 4 files


In [17]:
!jq . tmp/some-table/_delta_log/00000000000000000001.json

{
  "add": {
    "path": "1-57abb6fb-2249-43ba-a7be-cf09bcc230de-0.parquet",
    "size": 2204,
    "partitionValues": {},
    "modificationTime": 1701740386169,
    "dataChange": true,
    "stats": "{\"numRecords\": 2, \"minValues\": {\"num\": 8, \"letter\": \"dd\"}, \"maxValues\": {\"num\": 9, \"letter\": \"ee\"}, \"nullCount\": {\"num\": 0, \"letter\": 0}}"
  }
}
{
  "commitInfo": {
    "timestamp": 1701740386169,
    "operation": "WRITE",
    "operationParameters": {
      "partitionBy": "[]",
      "mode": "Append"
    },
    "clientVersion": "delta-rs.0.17.0"
  }
}


## Overwrite

In [18]:
df = pd.DataFrame({"num": [11, 22], "letter": ["aa", "bb"]})

In [19]:
write_deltalake("tmp/some-table", df, mode="overwrite")

In [20]:
dt = dl.DeltaTable("tmp/some-table")

In [21]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|    11 | aa       |
|    22 | bb       |
+-------+----------+


In [22]:
!tree tmp/some-table

tmp/some-table
├── 0-62dffa23-bbe1-4496-8fb5-bff6724dc677-0.parquet
├── 1-57abb6fb-2249-43ba-a7be-cf09bcc230de-0.parquet
├── 2-95ef2108-480c-4b89-96f0-ff9185dab9ad-0.parquet
└── _delta_log
    ├── 00000000000000000000.json
    ├── 00000000000000000001.json
    └── 00000000000000000002.json

1 directory, 6 files


In [23]:
!jq . tmp/some-table/_delta_log/00000000000000000002.json

{
  "add": {
    "path": "2-95ef2108-480c-4b89-96f0-ff9185dab9ad-0.parquet",
    "size": 2204,
    "partitionValues": {},
    "modificationTime": 1701740465102,
    "dataChange": true,
    "stats": "{\"numRecords\": 2, \"minValues\": {\"num\": 11, \"letter\": \"aa\"}, \"maxValues\": {\"num\": 22, \"letter\": \"bb\"}, \"nullCount\": {\"num\": 0, \"letter\": 0}}"
  }
}
{
  "remove": {
    "path": "0-62dffa23-bbe1-4496-8fb5-bff6724dc677-0.parquet",
    "deletionTimestamp": 1701740465102,
    "dataChange": true,
    "extendedFileMetadata": false,
    "partitionValues": {},
    "size": 2208
  }
}
{
  "remove": {
    "path": "1-57abb6fb-2249-43ba-a7be-cf09bcc230de-0.parquet",
    "deletionTimestamp": 1701740465102,
    "dataChange": true,
    "extendedFileMetadata": false,
    "partitionValues": {},
    "size": 2204
  }
}
{
  "commitInfo": {
    "timestamp": 1701740465102,
    "operation": "WRITE",
    "operationParameters": {
      "mode": "Overwrite",
      "partitionBy": "[]"
    },
    "

## Time travel to previous version

In [14]:
dt = DeltaTable("tmp/some-table", version=0)

In [15]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|     1 | a        |
|     2 | b        |
|     3 | c        |
+-------+----------+


In [16]:
# time travel to version 1

DeltaTable("tmp/some-table", version=1).to_pandas()

,num,letter
0,1,a
1,2,b
2,3,c
3,8,dd
4,9,ee


In [17]:
# it reads the latest version by default
DeltaTable("tmp/some-table").to_pandas()

,num,letter
0,11,aa
1,22,bb


## Schema enforcement

In [18]:
df = pd.DataFrame({"num": [5, 6], "animal": ["cat", "dog"]})

In [19]:
write_deltalake("tmp/some-table", df)

ValueError: Schema of data does not match table schema
Data schema:
num: int64
animal: string
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 474
Table Schema:
num: int64
letter: string

## Overwriting schema

In [21]:
write_deltalake("tmp/some-table", df, mode="overwrite", overwrite_schema=True)

In [23]:
dt = DeltaTable("tmp/some-table")

In [24]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | animal   |
|-------+----------|
|     5 | cat      |
|     6 | dog      |
+-------+----------+


## Merge

## Delete rows

In [29]:
df = pd.DataFrame({"num": [1, 2, 3, 4], "letter": ["a", "b", "c", "d"]})

In [31]:
dl.writer.write_deltalake("tmp/my-table", df)

In [32]:
dt = dl.DeltaTable("tmp/my-table")

In [33]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|     1 | a        |
|     2 | b        |
|     3 | c        |
|     4 | d        |
+-------+----------+


In [35]:
dt.delete("num > 2")

{'num_added_files': 1,
 'num_removed_files': 1,
 'num_deleted_rows': 2,
 'num_copied_rows': 2,
 'execution_time_ms': 8936,
 'scan_time_ms': 6637,
 'rewrite_time_ms': 2}

In [36]:
print(tabulate(dt.to_pandas(), headers="keys", tablefmt="psql", showindex=False))

+-------+----------+
|   num | letter   |
|-------+----------|
|     1 | a        |
|     2 | b        |
+-------+----------+
